<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

<!--NAVIGATION-->
< [Fast Fourier Transform Based Docking via ZDOCK](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta/blob/master/notebooks/07.01-Fast-Fourier-Transform-Based-Docking-via-ZDOCK.ipynb) | [Contents](toc.ipynb) |<p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta/blob/master/notebooks/07.02-Docking-Moves-in-Rosetta.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# Docking Moves in Rosetta

For the following exercises, download and clean the complex of colicin D and ImmD (1V74). Don't forget to import `pyrosetta` and initialize. Store three poses — a full-atom starting pose and centroid and full-atom “working” poses.

```
from pyrosetta import *
from pyrosetta.teaching import *
from pyrosetta.toolbox import cleanATOM

pyrosetta.init()
cleanATOM("1V74.pdb")
pose = pose_from_file("1V74.clean.pdb")
starting_pose = pose.clone()
cen_pose = pose.clone()
cen_switch = SwitchResidueTypeSetMover("centroid")
cen_switch.apply(cen_pose)
starting_cen_pose = cen_pose.clone()
```

In [15]:
### BEGIN SOLUTION
from pyrosetta import *
from pyrosetta.teaching import *
from pyrosetta.toolbox import cleanATOM

pyrosetta.init()
from pyrosetta.toolbox import pose_from_rcsb
pose = pose_from_rcsb("1V74")
starting_pose = pose.clone()
cen_pose = pose.clone()
cen_switch = SwitchResidueTypeSetMover("centroid")
cen_switch.apply(cen_pose)
starting_cen_pose = cen_pose.clone()
### END SOLUTION

core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python36.mac r208 2019.04+release.fd666910a5e fd666910a5edac957383b32b3b4c9d10020f34c1 http://www.pyrosetta.org 2019-01-22T15:55:37
core.init: command: PyRosetta -ex1 -ex2aro -database /Users/kathyle/Computational Protein Prediction and Design/PyRosetta4.Release.python36.mac.release-208/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=1234229970 seed_offset=0 real_seed=1234229970
core.init.random: RandomGenerator:init: Normal mode, seed=1234229970 RG_type=mt19937
core.import_pose.import_pose: File '1V74.clean.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue LEU:CtermProteinFull 107
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue LEU:CtermProteinFull 194


## Rigid-body transformations and Fold Trees

<img src="rigid-body.png" width=400>

The fundamental docking move is a rigid-body transformation consisting of a translation and rotation. Any rigid body move also needs to know which part moves and which part is fixed. In Rosetta, this division is known as a `Jump` and the set of protein segments and jumps are stored in an object attached to a pose called a `FoldTree`.

```
print(pose.fold_tree())
```

In [16]:
### BEGIN SOLUTION
print(pose.fold_tree())
### END SOLUTION

FOLD_TREE  EDGE 1 107 -1  EDGE 1 108 1  EDGE 108 194 -1 


In the `FoldTree` printout, each three number sequence following the word `EDGE` is the beginning and ending residue number, then a code. The codes are -1 for stretches of protein and any positive integer for a `Jump`, which represents the `Jump` number.

__Question:__ View the fold tree of your full-atom pose. How many `Jumps` are there in your pose?

In [17]:
### BEGIN SOLUTION
print(starting_pose.fold_tree())
print("There's 1 Jump!")
### END SOLUTION

FOLD_TREE  EDGE 1 107 -1  EDGE 1 108 1  EDGE 108 194 -1 
There's 1 Jump!


<img src="first-jump.png" width=400>

By default, there is a `Jump` between the N-terminus of chain A and the N-terminus of chain B, but we can change this using the exposed method `setup_foldtree()`.

```
from pyrosetta.rosetta.protocols.docking import setup_foldtree
print(pose.fold_tree())
setup_foldtree(pose, "A_B", Vector1([1]))
setup_foldtree(starting_pose, "A_B", Vector1([1]))
print(pose.fold_tree())
```

In [8]:
### BEGIN SOLUITON
from pyrosetta.rosetta.protocols.docking import setup_foldtree
print(pose.fold_tree())
setup_foldtree(pose, "A_B", Vector1([1]))
setup_foldtree(starting_pose, "A_B", Vector1([1]))
print(pose.fold_tree())
### END SOLUTION

FOLD_TREE  EDGE 1 107 -1  EDGE 1 108 1  EDGE 108 194 -1 
FOLD_TREE  EDGE 1 88 -1  EDGE 88 107 -1  EDGE 88 158 1  EDGE 158 108 -1  EDGE 158 194 -1 


The argument "A_B" tells Rosetta to make chain A the “rigid” chain and allow chain B to move. If there were more chains in the pdb structure, supplying "AB_C" would hold chains A and B rigid together as a single unit and allow chain C to move. (The third argument `Vector1([1]`) is required, it creates a Rosetta vector object — indexed from 1 — with one element that identifies the first `Jump` in the `FoldTree` for docking use.)

__Question:__ The above command changed the `FoldTree` and prepared it for docking. What has changed?

In [9]:
### BEGIN SOLUTION
print("We changed the Jump that was connecting the N-termini of A and B into a Jump that connects the centers of A and B.")
### END SOLUTION

We changed the Jump that was connecting the N-termini of A and B into a Jump that connects the centers of A and B.


<img src="second-jump.png" width=400>

You can see the type of information in the `Jump` by printing it from the `pose`:

```
jump_num = 1
print(pose.jump(jump_num).get_rotation())
print(pose.jump(jump_num).get_translation())
```

In [10]:
### BEGIN SOLUTION
jump_num = 1
print(pose.jump(jump_num).get_rotation())
print('\n')
print(pose.jump(jump_num).get_translation())
### END SOLUTION

    -0.7685217364647527     -0.5410492486350577     -0.3415260621572572
    -0.4152200273442840      0.8278586413578777     -0.3771503663279437
     0.4867922241299832     -0.1480397935709470     -0.8608818444165606



     -2.528204534753801      -11.32734702901771      -11.01155062262599


Write out the rotation matrix and the translation vector defined by the jump.

                       X |_________, _________, _________|
                       Y |_________, _________, _________|
                       Z |_________, _________, _________|
                        
                        (_________, _________, _________)

## RigidBody Movers

The two basic manipulations are translations and rotations. For translation, the change in x, y, and z coordinates are needed as well as the `Jump` number. A rotation requires a center and an axis about which to rotate. The rigid-body displacement can be altered directly with the `RigidBodyTransMover` for translations or the `RigidBodySpinMover` for rotations.


However, for structure prediction calculations, we have a `Mover` that is preconfigured to make random movements varying around set magnitudes (in this case, a mean of 8° rotation and 3 Å translation) located in the `rosetta.protocols.rigid` namespace, (which we will rename with an alias `rigid_moves` for our convenience) :

```
import pyrosetta.rosetta.protocols.rigid as rigid_moves
pert_mover = rigid_moves.RigidBodyPerturbMover(jump_num, 8, 3)
```

In [71]:
### BEGIN SOLUTION
import pyrosetta.rosetta.protocols.rigid as rigid_moves
pert_mover = rigid_moves.RigidBodyPerturbMover(jump_num, 8, 3)
### END SOLUTION

Apply the `RigidBodyPerturbMover` to a full-atom pose and use a `PyMOLMover` to confirm that the motions are what you expect.
__Remember to put in your IP address when using the PyMOL Mover__

__Question:__ What are the new rotation matrix and translation vector in the `Jump`? How many ångströms did the downstream protein move?

In [72]:
from pyrosetta import PyMOLMover
pymol = PyMOLMover()
pymol.apply(pose)

In [73]:
pert_mover.apply(pose)
pymol.apply(pose)

Global perturbations are useful for making completely randomized starting structures. The following `Mover` will rotate a protein about its geometric center. The final orientation is equally distributed over the “globe”.

```
randomize1 = rigid_moves.RigidBodyRandomizeMover(pose, jump_num, rigid_moves.partner_upstream)
randomize2 = rigid_moves.RigidBodyRandomizeMover(pose, jump_num, rigid_moves.partner_downstream)
```

(`partner_upstream` and `partner_downstream` are predefined terms within the `pyrosetta.rosetta.protocols.rigid` namespace, which in our case refer to chains A and B, respectively.)

In [74]:
### BEGIN SOLUTION
randomize1 = rigid_moves.RigidBodyRandomizeMover(pose, jump_num, rigid_moves.partner_upstream)
randomize2 = rigid_moves.RigidBodyRandomizeMover(pose, jump_num, rigid_moves.partner_downstream)
### END SOLUTION

Apply both `Movers` to the starting structure, and view the structure in PyMOL. (You might view it along with the original pose).

__Question:__ Does the new conformation look like a candidate docked structure yet?

In [85]:
### BEGIN SOLUTION
randomize1.apply(pose)
pymol.apply(pose)
### END SOLUTION

protocols.moves.RigidBodyMover: Randomize: Jump (before): RT -0.313379 -0.48962 0.813674 0.0290122 -0.861371 -0.507147 0.949185 -0.135323 0.28414 3.19217 8.40521 -27.6032
protocols.moves.RigidBodyMover: Randomize: Rot. Center (before): 8.41166 24.8844 13.8747
protocols.geometry.RB_geometry: random_reorientation_matrix phi: 75.3282 psi: 77.7212 theta: 114.406
protocols.moves.RigidBodyMover: Randomize: Jump (after):  RT 0.605281 -0.545896 -0.579339 0.630495 -0.115492 0.767553 -0.485913 -0.829855 0.274279 -10.7822 -23.0051 16.4798
protocols.moves.RigidBodyMover: Randomize: Rot. Center (after):  8.41166 24.8844 13.8747
protocols.moves.RigidBodyMover: Randomize: ---


In [84]:
### BEGIN SOLUTION
randomize2.apply(pose)
pymol.apply(pose)
### END SOLUTION

protocols.moves.RigidBodyMover: Randomize: Jump (before): RT 0.0162323 0.141197 0.989848 0.822283 0.561336 -0.0935564 -0.568847 0.815454 -0.106992 3.31095 8.54388 -22.2052
protocols.moves.RigidBodyMover: Randomize: Rot. Center (before): 16.007 46.4845 -2.711
protocols.geometry.RB_geometry: random_reorientation_matrix phi: 222.104 psi: 46.9095 theta: 110.52
protocols.moves.RigidBodyMover: Randomize: Jump (after):  RT -0.313379 -0.48962 0.813674 0.0290122 -0.861371 -0.507147 0.949185 -0.135323 0.28414 3.19217 8.40521 -27.6032
protocols.moves.RigidBodyMover: Randomize: Rot. Center (after):  16.007 46.4845 -2.711
protocols.moves.RigidBodyMover: Randomize: ---


Since proteins are not spherical, sometimes the random orientation creates severe clashes between the docking partners, and other times it places the partners so they are no longer touching. The `FaDockingSlideIntoContact` `Mover` will translate the downstream protein along the line of protein centers until the proteins are in contact.

```
slide = DockingSlideIntoContact(jump_num)  # for centroid mode
slide = FaDockingSlideIntoContact(jump_num)  # for full-atom mode
slide.apply(pose)
```

In [86]:
### BEGIN SOLUTION
slide = DockingSlideIntoContact(jump_num)  # for centroid mode
slide = FaDockingSlideIntoContact(jump_num)  # for full-atom mode
slide.apply(pose)
pymol.apply(pose)
### END SOLUTION

protocols.moves.RigidBodyMover: Translate: Jump (before): RT 0.605281 -0.545896 -0.579339 0.630495 -0.115492 0.767553 -0.485913 -0.829855 0.274279 -10.7822 -23.0051 16.4798
protocols.moves.RigidBodyMover: Translate: Jump (after):  RT 0.605281 -0.545896 -0.579339 0.630495 -0.115492 0.767553 -0.485913 -0.829855 0.274279 -10.2394 -21.3697 15.4646
protocols.moves.RigidBodyMover: Translate: Jump (before): RT 0.605281 -0.545896 -0.579339 0.630495 -0.115492 0.767553 -0.485913 -0.829855 0.274279 -10.2394 -21.3697 15.4646
protocols.moves.RigidBodyMover: Translate: Jump (after):  RT 0.605281 -0.545896 -0.579339 0.630495 -0.115492 0.767553 -0.485913 -0.829855 0.274279 -10.7822 -23.0051 16.4798
protocols.moves.RigidBodyMover: Translate: Jump (before): RT 0.605281 -0.545896 -0.579339 0.630495 -0.115492 0.767553 -0.485913 -0.829855 0.274279 -10.7822 -23.0051 16.4798
protocols.moves.RigidBodyMover: Translate: Jump (after):  RT 0.605281 -0.545896 -0.579339 0.630495 -0.115492 0.767553 -0.485913 -0.8298

The `MinMover`, which we have previously used to change torsion angles to find the nearest minimum in the score function, can also operate on the jump translation and rotation. It suffices to set the `Jump` variable as moveable in the `MoveMap`:

```
movemap = MoveMap()
movemap.set_jump(jump_num, True)
min_mover = MinMover()
min_mover.movemap(movemap)
scorefxn = get_fa_scorefxn()
min_mover.score_function(scorefxn)
```

In [87]:
### BEGIN SOLUTION
movemap = MoveMap()
movemap.set_jump(jump_num, True)
min_mover = MinMover()
min_mover.movemap(movemap)
scorefxn = get_fa_scorefxn()
min_mover.score_function(scorefxn)
### END SOLUTION

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/AccStrength.csv
basic.io.database: Database file opened: scoring/score_functions/rama/fd/all.ramaProb
basic.io.database: Database file opened: scoring/score_functions/rama/fd/prepro.ramaProb
basic.io.database: Database file opened: scoring/score_functions/omega/omega_ppdep.all.txt
basic.io.database: Database file opened: scoring/score_functions/omega/omega_ppdep.gly.txt
basic.io.database: Database file opened: scoring/score_functions/omega/ome

Apply the above `MinMover` to the working pose

```
scorefxn(pose)
min_mover.apply(pose)
print(pose.jump(jump_num).get_rotation())
print(pose.jump(jump_num).get_translation())
```

__Question:__ How much does the score change? What are the new rotation matrix and translation vector in the `Jump`? How many Ångstroms did the downstream protein move?

In [88]:
### BEGIN SOLUTION
scorefxn(pose)
min_mover.apply(pose)
print(pose.jump(jump_num).get_rotation())
print(pose.jump(jump_num).get_translation())
### END SOLUTION

basic.io.database: Database file opened: scoring/score_functions/elec_cp_reps.dat
core.scoring.elec.util: Read 40 countpair representative atoms
core.pack.dunbrack.RotamerLibrary: shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: shapovalov_lib::shap_dun10_smooth_level of 1( aka lowest_smooth ) got activated.
core.pack.dunbrack.RotamerLibrary: Binary rotamer library selected: /Users/kathyle/Computational Protein Prediction and Design/PyRosetta4.Release.python36.mac.release-208/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin
core.pack.dunbrack.RotamerLibrary: Using Dunbrack library binary file '/Users/kathyle/Computational Protein Prediction and Design/PyRosetta4.Release.python36.mac.release-208/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin'.
core.pack.dunbrack.RotamerLibrary: Dunbrack 2010 library took 0.63642 seconds to load from binary
     0.1149075345037018     -0.3772309955000518     -0.9189630213171396
   

## Low-Resolution Docking via RosettaDock


RosettaDock can also perform global docking runs, but it can require significant time. Typically, 105 to 106 decoys are needed in a global run. For this workshop, we will create a much smaller number and learn the tools needed to handle large runs.


Docking is available as a mover that completely encompasses the protocol. To use the `Mover`, you will need a starting pose with both chains and a jump defined. The structure must be in low-resolution (centroid) mode, and you will need the low-resolution docking score function:

```
scorefxn_low = create_score_function("interchain_cen")
```

In [89]:
### BEGIN SOLUTION
scorefxn_low = create_score_function("interchain_cen")
### END SOLUTION

Randomize your centroid version of the complex. Then, create low-resolution docking structures as follows:

```
dock_lowres = DockingLowRes(scorefxn_low, jump_num)
print(cen_pose.fold_tree())
setup_foldtree(cen_pose, "A_B", Vector1([1]))
print(cen_pose.fold_tree())
dock_lowres.apply(cen_pose)
```

In [18]:
### BEGIN SOLUTION
dock_lowres = DockingLowRes(scorefxn_low, jump_num)
print(cen_pose.fold_tree())
setup_foldtree(cen_pose, "A_B", Vector1([1]))
print(cen_pose.fold_tree())
dock_lowres.apply(cen_pose)
### END SOLUTION

NameError: name 'scorefxn_low' is not defined

You can compare structures by calculating the root-mean-squared deviation of all the Cα atoms, using the function `CA_rmsd(pose1, pose2)`. In docking, a more useful measure is the ligand RMSD, which is the deviation of the backbone Cα atoms of the ligand after superposition of the receptor protein backbones. You can calculate ligand RMSD with `calc_Lrmsd(pose1, pose2, Vector1([1])`).

```
print(CA_rmsd(cen_pose, starting_cen_pose))
print(calc_Lrmsd(cen_pose, starting_cen_pose, Vector1([1])))
```

__Question:__ Using both measures, how far did your pose move from the low-resolution search?

In [91]:
print(CA_rmsd(cen_pose, starting_cen_pose))
print(calc_Lrmsd(cen_pose, starting_cen_pose, Vector1([1])))

1.6642228364944458
4.431076531534755


Examine the created decoy in PyMOL.

__Question:__ Does it look like a reasonable structure for a protein-protein complex? Explain

In [99]:

pymol.keep_history(True)
pymol.apply(cen_pose)
pymol.apply(pose)

## Job Distributor


For exhaustive searches with Rosetta (docking, refinement, or folding), it is necessary to create a large number of candidate structures, termed “decoys”. This is often accomplished by spreading out the work over a large number of computers. Additionally, each decoy created needs to be individually labeled. The object that is responsible for managing the output is called a `PyJobDistributor`. Here, we will use a simple job distributor to create multiple structures. The following constructor sets the job distributor to create 10 decoys, with filenames like `output_1.pdb`, `output_2.pdb`, etc. The pdb files will also include scores according to the `ScoreFunction` provided.

```
jd = PyJobDistributor("output", 10, scorefxn_low)
```

In [100]:
jd = PyJobDistributor("output", 10, scorefxn_low)

Working on decoy: output_7.pdb


It is also useful to compare each decoy to the native structure (if it is known; otherwise any reference structure can be used). The job distributor will do the RMSD calculation and final scoring upon output. To set the native pose:

```
# your starting_cen_pose should be the native crystal structure
jd.native_pose = starting_cen_pose
```

In [101]:
# your starting_cen_pose should be the native crystal structure
jd.native_pose = starting_cen_pose

Create a randomized starting pose, working pose, fold tree, score function, job distributor, and low-resolution docking mover. Now, run the low-resolution docking protocol to create a structure, and output a decoy:

```
cen_pose.assign(starting_cen_pose)
dock_lowres.apply(cen_pose)
jd.output_decoy(cen_pose)
```

Do this twice and confirm that you have two output files in your working directory.

In [102]:
cen_pose.assign(starting_cen_pose)
dock_lowres.apply(cen_pose)
jd.output_decoy(cen_pose)

protocols.docking.DockingLowRes: in DockingLowRes.apply
protocols.docking.DockingLowRes: ////////////////////////////////////////////////////////////////////////////////
protocols.docking.DockingLowRes: ///                       Docking Low Res Protocol                           ///
protocols.docking.DockingLowRes: ///                                                                          ///
protocols.docking.DockingLowRes: /// Centroid Inner Cycles: 50                                                ///
protocols.docking.DockingLowRes: /// Centroid Outer Cycles: 10                                                ///
protocols.docking.DockingLowRes: /// Scorefunction:                                                           ///
protocols.docking.DockingLowRes: ScoreFunction::show():
weights: (interchain_pair 1) (interchain_vdw 1) (interchain_env 1) (interchain_contact 2)
energy_method_options: EnergyMethodOptions::show: aa_composition_setup_files:
protocols.docking.DockingLowRes: Ene

protocols.docking.DockingLowRes: HBondOptions::show: use_sp2_chi_penalty_: true
protocols.docking.DockingLowRes: HBondOptions::show: sp2_BAH180_rise_: 0.75
protocols.docking.DockingLowRes: HBondOptions::show: sp2_outer_width_: 0.357
protocols.docking.DockingLowRes: HBondOptions::show: measure_sp3acc_BAH_from_hvy_: true
protocols.docking.DockingLowRes: HBondOptions::show: fade_energy_: 1
protocols.docking.DockingLowRes: HBondOptions::show: exclude_ether_oxygens_: 0
protocols.docking.DockingLowRes: HBondOptions::show: Mbhbond: false
protocols.docking.DockingLowRes: HbondOptions::show: mphbond: false
protocols.docking.DockingLowRes: HBondOptions::show: hbond_energy_shift: 0
protocols.docking.DockingLowRes: HBondOptions::show: water_hybrid_sf: false
protocols.docking.DockingLowRes: RNA_EnergyMethodOptions::show: syn_G_potential_bonus: 0
protocols.docking.DockingLowRes: RNA_EnergyMethodOptions::show: torsion_potential: ps_04282011
protocols.docking.DockingLowRes: RNA_EnergyMethodOptions::sh

protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.

protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Acceptance rate is: 0.06
protocols.docking.

protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Acceptance rate is: 0.06
protocols.docking.DockingLowRes: Translation magnitude: 0.372009
protocols.docking.DockingLowRes: Rotation magnitude: 2.65721
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.Dockin

protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Acceptance rate is: 0.14
protocols.docking.DockingLowRes: Translation magnitude: 0.301327
protocols.docking.DockingLowRes: Rotation magnitude: 2.15234
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.Dockin

Whenever the `output_decoy()` method is called, the `current_num` variable of the `PyJobDistributor` is incremented, and it also outputs an updated score file: `output.fasc`. We can finish the set of 10 decoys by using the `PyJobDistributor` to set up a loop:

```
while not jd.job_complete:
    cen_pose.assign(starting_cen_pose)
    dock_lowres.apply(cen_pose)
    jd.output_decoy(cen_pose)
```

Note the `jd.job_complete` Boolean variable that indicates whether all 10 decoys have been created.

In [103]:
while not jd.job_complete:
    cen_pose.assign(starting_cen_pose)
    dock_lowres.apply(cen_pose)
    jd.output_decoy(cen_pose)

protocols.docking.DockingLowRes: in DockingLowRes.apply
protocols.docking.DockingLowRes: ////////////////////////////////////////////////////////////////////////////////
protocols.docking.DockingLowRes: ///                       Docking Low Res Protocol                           ///
protocols.docking.DockingLowRes: ///                                                                          ///
protocols.docking.DockingLowRes: /// Centroid Inner Cycles: 50                                                ///
protocols.docking.DockingLowRes: /// Centroid Outer Cycles: 10                                                ///
protocols.docking.DockingLowRes: /// Scorefunction:                                                           ///
protocols.docking.DockingLowRes: ScoreFunction::show():
weights: (interchain_pair 1) (interchain_vdw 1) (interchain_env 1) (interchain_contact 2)
energy_method_options: EnergyMethodOptions::show: aa_composition_setup_files:
protocols.docking.DockingLowRes: Ene

protocols.docking.DockingLowRes: HBondOptions::show: use_sp2_chi_penalty_: true
protocols.docking.DockingLowRes: HBondOptions::show: sp2_BAH180_rise_: 0.75
protocols.docking.DockingLowRes: HBondOptions::show: sp2_outer_width_: 0.357
protocols.docking.DockingLowRes: HBondOptions::show: measure_sp3acc_BAH_from_hvy_: true
protocols.docking.DockingLowRes: HBondOptions::show: fade_energy_: 1
protocols.docking.DockingLowRes: HBondOptions::show: exclude_ether_oxygens_: 0
protocols.docking.DockingLowRes: HBondOptions::show: Mbhbond: false
protocols.docking.DockingLowRes: HbondOptions::show: mphbond: false
protocols.docking.DockingLowRes: HBondOptions::show: hbond_energy_shift: 0
protocols.docking.DockingLowRes: HBondOptions::show: water_hybrid_sf: false
protocols.docking.DockingLowRes: RNA_EnergyMethodOptions::show: syn_G_potential_bonus: 0
protocols.docking.DockingLowRes: RNA_EnergyMethodOptions::show: torsion_potential: ps_04282011
protocols.docking.DockingLowRes: RNA_EnergyMethodOptions::sh

protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.

protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Acceptance rate is: 0.08
protocols.docking.

protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Acceptance rate is: 0.06
protocols.docking.DockingLowRes: Translation magnitude: 0.372009
protocols.docking.DockingLowRes: Rotation magnitude: 2.65721
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.Dockin

...

protocols.docking.DockingLowRes: /// Centroid Inner Cycles: 50                                                ///
protocols.docking.DockingLowRes: /// Centroid Outer Cycles: 10                                                ///
protocols.docking.DockingLowRes: /// Scorefunction:                                                           ///
protocols.docking.DockingLowRes: ScoreFunction::show():
weights: (interchain_pair 1) (interchain_vdw 1) (interchain_env 1) (interchain_contact 2)
energy_method_options: EnergyMethodOptions::show: aa_composition_setup_files:
protocols.docking.DockingLowRes: EnergyMethodOptions::show: mhc_epitope_setup_files:
protocols.docking.DockingLowRes: EnergyMethodOptions::show: netcharge_setup_files:
protocols.docking.DockingLowRes: EnergyMethodOptions::show: aspartimide_penalty_value: 25
protocols.docking.DockingLowRes: EnergyMethodOptions::show: etable_type: FA_STANDARD_DEFAULT
analytic_etable_evaluation: 1
EnergyMethodOptions::show: method_weights: free_res
E

protocols.docking.DockingLowRes: HBondOptions::show: fade_energy_: 1
protocols.docking.DockingLowRes: HBondOptions::show: exclude_ether_oxygens_: 0
protocols.docking.DockingLowRes: HBondOptions::show: Mbhbond: false
protocols.docking.DockingLowRes: HbondOptions::show: mphbond: false
protocols.docking.DockingLowRes: HBondOptions::show: hbond_energy_shift: 0
protocols.docking.DockingLowRes: HBondOptions::show: water_hybrid_sf: false
protocols.docking.DockingLowRes: RNA_EnergyMethodOptions::show: syn_G_potential_bonus: 0
protocols.docking.DockingLowRes: RNA_EnergyMethodOptions::show: torsion_potential: ps_04282011
protocols.docking.DockingLowRes: RNA_EnergyMethodOptions::show: suiteness_bonus: Richardson
protocols.docking.DockingLowRes: RNA_EnergyMethodOptions::show: rna_base_pair_xy_filename: scoring/rna/rna_base_pair_xy.dat
protocols.docking.DockingLowRes: FreeDOF_Options::show: free_suite_bonus: -1
protocols.docking.DockingLowRes: FreeDOF_Options::show: free_2HOprime_bonus: -0.5
protoc

protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.

protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Acceptance rate is: 0.12
protocols.docking.DockingLowRes: Translation magnitude: 0.372009
protocols.docking.DockingLowRes: Rotation magnitude: 2.65721
protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.Dockin

protocols.docking.DockingLowRes: Translation magnitude: 0.301327
protocols.docking.DockingLowRes: Rotation magnitude: 2.15234
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.dockin

Run the loop to create 10 structures. The score file, `output.fasc` summarizes the energies and RMSDs of all structures created.

__Question:__ Examine that file. What is the lowest score? What is the lowest energy?

Reset the `PyJobDistributor` to create 100 decoys (or more or less, as the speed of your processor allows) by reconstructing it. Rerun the loop above to make 100 decoys. Use your score file to plot score versus RMSD. (Two easy ways to do this are to import the score file into Excel or to use the Linux command gnuplot.)

```
jd = PyJobDistributor("output", 100, scorefxn_low)
while not jd.job_complete:
    cen_pose.assign(starting_cen_pose)
    dock_lowres.apply(cen_pose)
    jd.output_decoy(cen_pose)
```

__Question:__ Do you see an energy funnel?

In [ ]:
jd = PyJobDistributor("output", 100, scorefxn_low)
while not jd.job_complete:
    cen_pose.assign(starting_cen_pose)
    dock_lowres.apply(cen_pose)
    jd.output_decoy(cen_pose)

Working on decoy: output_62.pdb
protocols.docking.DockingLowRes: in DockingLowRes.apply
protocols.docking.DockingLowRes: ////////////////////////////////////////////////////////////////////////////////
protocols.docking.DockingLowRes: ///                       Docking Low Res Protocol                           ///
protocols.docking.DockingLowRes: ///                                                                          ///
protocols.docking.DockingLowRes: /// Centroid Inner Cycles: 50                                                ///
protocols.docking.DockingLowRes: /// Centroid Outer Cycles: 10                                                ///
protocols.docking.DockingLowRes: /// Scorefunction:                                                           ///
protocols.docking.DockingLowRes: ScoreFunction::show():
weights: (interchain_pair 1) (interchain_vdw 1) (interchain_env 1) (interchain_contact 2)
energy_method_options: EnergyMethodOptions::show: aa_composition_setup_files:
prot

protocols.docking.DockingLowRes: HBondOptions::show: params_database_tag_: ref2015_params
protocols.docking.DockingLowRes: HBondOptions::show: use_sp2_chi_penalty_: true
protocols.docking.DockingLowRes: HBondOptions::show: sp2_BAH180_rise_: 0.75
protocols.docking.DockingLowRes: HBondOptions::show: sp2_outer_width_: 0.357
protocols.docking.DockingLowRes: HBondOptions::show: measure_sp3acc_BAH_from_hvy_: true
protocols.docking.DockingLowRes: HBondOptions::show: fade_energy_: 1
protocols.docking.DockingLowRes: HBondOptions::show: exclude_ether_oxygens_: 0
protocols.docking.DockingLowRes: HBondOptions::show: Mbhbond: false
protocols.docking.DockingLowRes: HbondOptions::show: mphbond: false
protocols.docking.DockingLowRes: HBondOptions::show: hbond_energy_shift: 0
protocols.docking.DockingLowRes: HBondOptions::show: water_hybrid_sf: false
protocols.docking.DockingLowRes: RNA_EnergyMethodOptions::show: syn_G_potential_bonus: 0
protocols.docking.DockingLowRes: RNA_EnergyMethodOptions::show: t

protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.

protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.

protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Acceptance rate is: 0.02
protocols.docking.DockingLowRes: Translation magnitude: 0.301327
protocols.docking.DockingLowRes: Rotation magnitude: 2.15234
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.Dockin

...

protocols.docking.DockingLowRes: HBondOptions::show: params_database_tag_: ref2015_params
protocols.docking.DockingLowRes: HBondOptions::show: use_sp2_chi_penalty_: true
protocols.docking.DockingLowRes: HBondOptions::show: sp2_BAH180_rise_: 0.75
protocols.docking.DockingLowRes: HBondOptions::show: sp2_outer_width_: 0.357
protocols.docking.DockingLowRes: HBondOptions::show: measure_sp3acc_BAH_from_hvy_: true
protocols.docking.DockingLowRes: HBondOptions::show: fade_energy_: 1
protocols.docking.DockingLowRes: HBondOptions::show: exclude_ether_oxygens_: 0
protocols.docking.DockingLowRes: HBondOptions::show: Mbhbond: false
protocols.docking.DockingLowRes: HbondOptions::show: mphbond: false
protocols.docking.DockingLowRes: HBondOptions::show: hbond_energy_shift: 0
protocols.docking.DockingLowRes: HBondOptions::show: water_hybrid_sf: false
protocols.docking.DockingLowRes: RNA_EnergyMethodOptions::show: syn_G_potential_bonus: 0
protocols.docking.DockingLowRes: RNA_EnergyMethodOptions::show: t

protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Accepted
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.

protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.

protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Acceptance rate is: 0.04
protocols.docking.DockingLowRes: Translation magnitude: 0.372009
protocols.docking.DockingLowRes: Rotation magnitude: 2.65721
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.Dockin

protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Acceptance rate is: 0.06
protocols.docking.DockingLowRes: Translation magnitude: 0.301327
protocols.docking.DockingLowRes: Rotation magnitude: 2.15234
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.DockingLowRes: Rigid Body Perturbation Rejected
protocols.docking.Dockin

## High-Resolution Docking


The high-resolution stage of RosettaDock is also available as a `Mover`. This mover encompasses random rigid-body moves, side-chain packing, and gradient-based minimization in the rigid-body coordinates. High-resolution docking needs an all-atom score function. The optimized docking weights are available as a patch to the standard all-atom energy function.

```
scorefxn_high = create_score_function("ref2015.wts", "docking")
dock_hires = DockMCMProtocol()
dock_hires.set_scorefxn(scorefxn_high)
dock_hires.set_partners("A_B")  # make sure the FoldTree is set up properly
```

__Note__ that unlike for `DockingLowRes`, we must supply the docking partners with `"A_B"` instead of `jump_num`.

In [ ]:
scorefxn_high = create_score_function("ref2015.wts", "docking")
dock_hires = DockMCMProtocol()
dock_hires.set_scorefxn(scorefxn_high)
dock_hires.set_partners("A_B")  # make sure the FoldTree is set up properly


A high-resolution decoy needs side chains. One way to place the side chains is to call the `PackMover`, which will generate a conformation from rotamers. A second way is to copy the side chains from the original monomer structures. This is often helpful for docking calculations since the monomer crystal structures have good side chain positions.

```
recover_sidechains = ReturnSidechainMover(starting_pose)
recover_sidechains.apply(pose)
```

In [ ]:
recover_sidechains = ReturnSidechainMover(starting_pose)
recover_sidechains.apply(pose)

Load one of your low-resolution decoys, add the side chains from the starting pose, and refine the decoy using high-resolution docking.

__Question:__ How far did the structure move during refinement? How much did the score improve?

Starting from your lowest-scoring low-resolution decoy, create three high-resolution decoys. (You might use the `PyJobDistributor`.) Do the same starting from the native structure.

__Questions:__

- How do the refined-native scores compare to the refined-decoy scores?

- What is the RMSD of the refined native? Why is it not zero?

- How much variation do you see in the refined native scores? In the refined decoy scores? Is the difference between the refined natives and the refined decoys significant?

## Docking Funnel

Using a `PyJobDistributor` and `DockMCMProtocol`, create 10 decoys starting with a `RigidBodyRandomizeMover` perturbation of `partner_downstream`, 10 decoys starting from different local random perturbations (8°, 3 Å), 10 decoys starting from low-resolution decoys, and 10 starting from the native structure. Plot all of these points on a funnel plot.

__Question:__ How is the sampling from each method? Does the scoring function discriminate good complexes?

## Programming Exercises


- Output a structure with a 10 Å translation and another with a 30° rotation (both starting from the same starting structure), and load them into PyMOL to confirm the motions are what you expect.


- Diffusion. Make a series of random rigid body perturbations and record the RMSD after each. Plot RMSD versus the number of moves. Does this process emulate diffusion? If it did, how would you know? (Hint: there is a way to plot these data to make them linear.)


- Starting from a low-resolution docking decoy, refine the structure in three separate ways:

    - side-chain packing
    
    - gradient-based minimization in the rigid-body coordinates

    - gradient-based minimization in the torsional coordinates

    - the docking high-resolution protocol

For each, note the change in RMSD and the change in score. Which operations move the protein the most? Which make the most difference in the score?
    
- Using the `MonteCarlo` object, the `RigidBodyMover`, `PackRotamers`, and the `MinMover`, create your own high-resolution docking protocol. Bonus: Can you tune it to beat the standard protocol? “Beating” the standard protocol could mean achieving lower energies, running in faster time, and/or being able to better predict complexes.

<!--NAVIGATION-->
< [Fast Fourier Transform Based Docking via ZDOCK](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta/blob/master/notebooks/07.01-Fast-Fourier-Transform-Based-Docking-via-ZDOCK.ipynb) | [Contents](toc.ipynb) |<p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta/blob/master/notebooks/07.02-Docking-Moves-in-Rosetta.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>